In [7]:
from dotenv import load_dotenv
import json
import os
import pandas as pd
import requests

In [12]:
load_dotenv()
EZRF_API_KEY = os.getenv('EZRF_API_KEY')
CHAIN_MAPPINGS = json.load(open('data/chains.json', 'r'))

In [13]:
url = 'https://ezrf-impact.vercel.app/api/trpc/projects.list?input=%7B%22json%22%3A%7B%22limit%22%3A1000%2C%22cursor%22%3A0%7D%7D'
headers = {
    'content-type': 'application/json',
    'round-id': 'the-sunnys',
    'x-api-key': EZRF_API_KEY
}
response = requests.get(url, headers=headers)
if response.status_code == 200:
    data = response.json()
    print("Data fetched successfully!")
else:
    print(f"Failed to fetch data. Status code: {response.status_code}")

Data fetched successfully!


In [14]:
applications = data['result']['data']['json']
print(len(applications), "applications received so far.")

1000 applications received so far.


In [26]:
def process_application(app):
    
    artifact_type = []
    address = []
    chain = []
    
    profile = app.get('profile', {})
    if not profile:
        profile = {}
    metadata = app.get('metadata', {})
    awards = metadata.get('sunnyAwards', {})
    contracts = awards.get('contracts', [])
    
    for contract in contracts:
        artifact_type.append('contract')
        address.append(contract.get('address', 'N/A'))
        chain_id = contract.get('chainId')
        chain.append(CHAIN_MAPPINGS.get(chain_id, 'All Superchain'))

    minting_wallet_address = awards.get('mintingWalletAddress', '')
    if minting_wallet_address:
        artifact_type.append('mintingWallet')
        address.append(minting_wallet_address)
        chain.append('All Superchain') 

    app_data = {
        'id': app['id'],
        'attester': app['attester'],
        'recipient': app['recipient'],
        'time': app['time'],
        'name': app['name'],
        'schemaId': app['schemaId'],
        'status': app['status'],
        'round': app['round'],
        'profile_name': profile.get('name', ''),
        'profile_image': profile.get('profileImageUrl', ''),
        'profile_banner': profile.get('bannerImageUrl', ''),
        'metadata_name': metadata.get('name', ''),
        'metadata_bio': metadata.get('bio', ''),
        'metadata_website': metadata.get('websiteUrl', ''),
        'sunnyAwards_project_type': awards.get('projectType', ''),
        'sunnyAwards_category': awards.get('category', ''),
        'sunnyAwards_category_details': awards.get('categoryDetails', ''),
        'sunnyAwards_avatar_url': awards.get('avatarUrl', ''),
        'sunnyAwards_cover_image_url': awards.get('coverImageUrl', ''),
        'artifact_type': artifact_type,
        'address': address,
        'chain': chain
    }
    
    return app_data

normalized_data = []
for (i,app) in enumerate(applications):
    try:
        normalized_app = process_application(app)
        normalized_data.append(normalized_app)
    except Exception as e:
        print(f"Exception at index {i}:", e)
        break

Exception at index 994: 'NoneType' object has no attribute 'get'


In [27]:
applications[994]

{'id': '0xf31bcac8d2efd1bc2ad7ca79a05218f90b789c19836955daf884eb1875a3852b',
 'refUID': '0x0000000000000000000000000000000000000000000000000000000000000000',
 'attester': '0x8Bc704386DCE0C4f004194684AdC44Edf6e85f07',
 'recipient': '0x61b84CaB9dd4B629cC934Bb2D1D69f9Dc43f0d34',
 'revoked': False,
 'schemaId': '0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4b4abdcdfda3b1d1b9061',
 'time': 1724774711,
 'name': 'Superchain Gods 🔴🔵',
 'metadataPtr': 'https://cdn.charmverse.io/attestations/0xf8757b1e38ff1b0c1893e47f7d815367332bec28fea4b4abdcdfda3b1d1b9061/project-4445a07c-3b7f-4b9e-be16-10a5ecab376b/metadata.json',
 'metadataType': {'type': 'BigNumber', 'hex': '0x00'},
 'type': '0x6170706c69636174696f6e000000000000000000000000000000000000000000',
 'round': '0x636d30617975733335303030357a777962347674757265753100000000000000',
 'uuid': '4445a07c-3b7f-4b9e-be16-10a5ecab376b',
 'profile': None,
 'metadata': None,
 'status': 'pending'}